Test 1: Import LLM

In [4]:
import os
os.environ["COHERE_API_KEY"] = ""

from langchain_cohere import ChatCohere #!pip install langchain-cohere
llm = ChatCohere(model='command-r', temperature=0)

In [5]:
llm.invoke("Hi, I'm Bob.")

AIMessage(content="Hi Bob! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '461b996c-aa2a-4ef7-aa2d-2c7c1e3f1ace', 'token_count': {'input_tokens': 72, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '461b996c-aa2a-4ef7-aa2d-2c7c1e3f1ace', 'token_count': {'input_tokens': 72, 'output_tokens': 16}}, id='run-469a7b07-1984-499d-b9cb-c932c8988f25-0')

In [ ]:
#TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}

In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = ""
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature = 0)
result = llm.invoke("Hello, I'm Bob?")
print(result.content)

Hi Bob, how can I help you today?


Test 2: Analyse, prepare and clean extracted data (mainly done in datacleaning-ver3.ipynb)

Test 3: Prepare LangChain documents

In [3]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader (file_path = './decoded_sacom.csv', encoding="utf-8")
data = loader.load()

In [4]:
print ("Number of documents:"+str(len(data)))
data[0]

Number of documents:14378


Document(page_content='Org ID: 193932\nOrg Name: RSL Ardrossan Sub Branch\nAKA: Ardrossan RSL; Returned & Services League Ardrossan\nAcronym: \nFormer Name: \nS Street Addr 1: RSL Hall, West Tce\nS Street Addr 2: \nS Suburb: Ardrossan\nS State: South Australia\nS Postcode: 5571\nPhone: 08 8837 3596\nMobile: \nEmail: ardrossan@rslsa.org.au\nWebsite: http://rslsa.org.au/stores/ardrossan\nOpen Hours: \nWheelchair Access: \nToilets Access: \nDisabled Parking: \nServices: Welfare and pensions support for ex-servicemen and their families\nSocial and recreational activities\nCommemoration activities - ANZAC Day, Remembrance Day and other significant events\nOrg Type: Business\nLocal Community dir: Service Clubs\nAdelaide Hills dir: \nOnkaparinga dir: \nSubjects: Ex-Defence Service Groups; Halls For Hire; Social & Activity Groups; Support & Resource Groups; Veterans\nPrimary Category: Recreation\nCouncil: Yorke Peninsula Council', metadata={'source': './decoded_sacom.csv', 'row': 0})

Test 4: Import embedding tool

In [7]:
#!pip install gpt4all
from langchain_community.embeddings import GPT4AllEmbeddings
embeddings = GPT4AllEmbeddings()
embed_text = embeddings.embed_query("hello, world!")
print(len(embed_text))
embed_text[0:3]

384


[-0.03814070299267769, 0.0328650176525116, -0.0055811433121562]

Test 5: Create FAISS vector database with embeddings and save to local directory

In [ ]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents (data, embeddings)
faiss.save_local("faiss_index")

Test 6: Load FAISS vector database from local directory and perform similarity search

In [8]:
from langchain_community.vectorstores import FAISS
faiss = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
faiss.similarity_search("hall for hire in prospect")

[Document(page_content='Org ID: 197897 \nOrg Name: Prospect Town Hall  \nAKA: Eliza Hall \nServices: Venue and meeting rooms for hire \nOrg Type: Government \nLocal Community dir: Halls & Venues for Hire \nSubjects: Halls For Hire; Meeting Rooms \nPrimary Category: Community Organisation & Development', metadata={'location': 'prospect', 'council': 'city of prospect'}),
 Document(page_content='Org ID: 198278 \nOrg Name: Lobethal Centennial Hall Inc \nServices: Hall for Hire\nVenue for Hire \nOrg Type: Community \nLocal Community dir: Halls & Venues for Hire \nSubjects: Cinemas; Halls For Hire; Hire \nPrimary Category: Community Organisation & Development', metadata={'location': 'lobethal', 'council': 'adelaide hills council'}),
 Document(page_content='Org ID: 200367 \nOrg Name: Roseworthy Soldiers Memorial Hall \nServices: Hall for hire\nAvailable for hire at hourly rates for meetings and other\nAvailable to hire for functions \nOrg Type: Community \nLocal Community dir: Halls & Venues 

In [9]:
retriever = faiss.as_retriever(search_type="similarity", search_kwargs={"k": 10})

Test 7: Link query, database, retriever and LLM to generate response

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
template = """You are an customer service assistant for SAcommunity.org.
Answer the question based only on the following context delimited by triple backtick.
context:```{context}```
If you cannot find the answer, tell them to be more precise.
You should reply in a friendly way.
Your must provide the website link in the following format:
"Org Name" "https://sacommunity.org/"+"org/"+"Org ID"
Question: {question}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
for chunk in rag_chain.stream("I want to quit smoking"):
    print(chunk, end="", flush=True)

Quitline might be a good first point of contact for you. They offer a confidential telephone information and counselling service for people who want to give up smoking. You can find them at https://sacommunity.org/org/202205.

Lung Foundation Australia also has lots of resources and information available, as well as peer support groups. They might have some useful services and recommendations too. You can find them at https://sacommunity.org/org/202421.

If you're interested in support groups more broadly, you could also check out the Millicent Respiratory Support Group. See https://sacommunity.org/org/227963 for details.